In [1]:
import numpy as np
import onnxruntime as ort
from PIL import Image
from urllib import request
from io import BytesIO

In [2]:
def download_image(url: str) -> Image.Image:
    """Download image from a URL."""
    with request.urlopen(url) as resp:
        buffer = resp.read()
    img = Image.open(BytesIO(buffer))
    return img

In [3]:
def prepare_image(img: Image.Image, target_size=(200,200)) -> Image.Image:
    """Resize image and ensure RGB mode."""
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img

In [4]:
def preprocess(img: Image.Image) -> np.ndarray:
    """Convert image to ONNX-ready numpy array."""
    x = np.array(img).astype("float32") / 255.0
    x = (x - 0.5) * 2            # scale to [-1,1]
    x = np.transpose(x, (2, 0, 1))  # HWC → CHW
    x = np.expand_dims(x, 0)        # add batch dimension
    return x

In [10]:
session = ort.InferenceSession("hair_classifier_v1.onnx")
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
print(output_name)

output


In [12]:
img_url = "https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg"
model_shape = session.get_inputs()[0].shape
target_height = int(model_shape[2])
target_width = int(model_shape[3])

img = download_image(img_url)
img = prepare_image(img, target_size=(target_height, target_width))
print((target_height, target_width))

(200, 200)


In [14]:
x = preprocess(img)
first_pixel_r = x[0,0,0,0]
print(first_pixel_r)

-0.52156866


In [15]:
pred = session.run([output_name], {input_name: x})[0]
score = float(pred[0][0])
print(score)

-0.203227236866951
